# 데이터 불러오기

In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import ast
import plotly.graph_objects as go
import plotly.express as px
from pyvis.network import Network
import plotly.colors

import dash
from dash import html, dcc
import plotly.express as px
from dash.dependencies import Input, Output


In [3]:
df = pd.read_excel('5.5_viz_dataset.xlsx')

In [5]:
# 리스트 형태의 문자열을 실제 리스트로 변환
df['actor_main_name'] = df['actor_main_name'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
df['actor_sub_name'] = df['actor_sub_name'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
df['actor_cameo_name'] = df['actor_cameo_name'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

In [6]:
df.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [7]:
# 매출액 및 스크린수 데이터 정제 (숫자 형태로 변환)
df['매출액'] = df['매출액'].replace(',', '', regex=True).astype(float)
df['스크린수'] = df['스크린수'].replace(',', '', regex=True).astype(int)

# 개봉일을 datetime 객체로 변환
df['개봉일'] = pd.to_datetime(df['개봉일'], format='%Y.%m.%d')

# 개봉일에서 년, 월, 일 추출하여 새로운 열 생성
df['년도'] = df['개봉일'].dt.year
df['월'] = df['개봉일'].dt.month
df['일'] = df['개봉일'].dt.day


In [8]:
# 새로운 열인 '년대' 생성
# '년대' 열은 주어진 '년도' 열을 기반으로 1990년대, 2000년대, 2010년대, 2020년대로 나눕니다.
def assign_decade(year):
    if year < 2000:
        return '1990년대'
    elif 2000 <= year < 2010:
        return '2000년대'
    elif 2010 <= year < 2020:
        return '2010년대'
    elif year >= 2020:
        return '2020년대'

df['년대'] = df['년도'].apply(assign_decade)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   순위                499 non-null    int64         
 1   영화명               499 non-null    object        
 2   개봉일               499 non-null    datetime64[ns]
 3   매출액               488 non-null    float64       
 4   관객수               499 non-null    int64         
 5   스크린수              499 non-null    int64         
 6   배급사               493 non-null    object        
 7   coach             499 non-null    object        
 8   actor_main_name   499 non-null    object        
 9   actor_sub_name    499 non-null    object        
 10  actor_cameo_name  499 non-null    object        
 11  maker             499 non-null    object        
 12  only_gener        499 non-null    object        
 13  genre_one         499 non-null    object        
 14  year              499 non-

# 동은님 요청 사항 : 
1. 연도별로 매출 및 관객 그래프(각 장르별로 얼마나 차지하는지도 포함)
이유. : 어느 해에 관객이 많았는지, 관객이 적었음에도 충분히 매출이 나올 수 있을지.
2. 월별 장르별 총 관객수, 월별 각 장르의 영화 출품 수
어느 월에 어떤 장르의 영화가 잘 먹히는지(실제로 그 월에 잘 내고 있는지 확인)


#### 각 장르에 대한 색상 코드 지정
genre_colors = {
    '사극': '#E6194B', '코미디': '#3CB44B', '판타지': '#FFE119', '드라마': '#4363D8',
    '액션': '#F58231', 'SF': '#911EB4', '범죄': '#46F0F0', '전쟁': '#F032E6',
    '미스터리': '#BCF60C', '어드벤처': '#FABEBE', '서부극(웨스턴)': '#008080',
    '멜로/로맨스': '#E6BEFF', '스릴러': '#9A6324', '다큐멘터리': '#FFFAC8',
    '공포(호러)': '#800000', '애니메이션': '#AAFFC3', '기타': '#808000', '뮤지컬': '#FFD8B1'
}

#### 예시 출력을 위해 색상 코드를 출력
for genre, color in genre_colors.items():
    print(f"{genre}: {color}")


# 5/6 그래프 그리기


내가 그리고 싶은 그래프 요청 사항이야
1) 연도별로 매출 및 관객 그래프(색상에는 각 장르별 정보를 담아줘)
원하는 정보 : 어느 해에 관객이 많았는지, 관객이 적었음에도 충분히 매출이 나올 수 있을지.
2) 월별 장르별 총 관객수, 월별 각 장르의 영화 출품 수(색상에는 각 장르별 정보를 담아줘)
원하는 정보 : 어느 월에 어떤 장르의 영화가 잘 먹히는지(실제로 그 월에 잘 내고 있는지 확인)

그래프 그리기 전에 어떤 그래프 그리면 좋을지 너가 후보지를 결정해서 나한테 물어봐줘

## 1. 연도별로 매출 및 관객 그래프(각 장르별로 얼마나 차지하는지도 포함)
이유. : 어느 해에 관객이 많았는지, 관객이 적었음에도 충분히 매출이 나올 수 있을지.

### 기본 그래프

In [10]:
import plotly.graph_objects as go
import pandas as pd

# 데이터 준비
df['년도'] = pd.to_datetime(df['개봉일']).dt.year  # '개봉일'이 datetime 타입인지 확인

# 연도별 관객 수 계산
yearly_audience = df.groupby('년도')['관객수'].sum().reset_index()

# 그래프 생성
fig = go.Figure()

# 막대 그래프 추가
fig.add_trace(go.Bar(
    x=yearly_audience['년도'],
    y=yearly_audience['관객수'],
    marker_color='blue'  # 막대 색상 설정
))

# 레이아웃 설정
fig.update_layout(
    title='연도별 총 관객 수',
    xaxis=dict(title='년도'),
    yaxis=dict(title='관객 수'),
    xaxis_tickangle=-45  # X축 레이블 각도 조정
)

# 그래프 표시
fig.show()

In [11]:
import plotly.graph_objects as go
import pandas as pd

# 데이터 준비
df['년도'] = pd.to_datetime(df['개봉일']).dt.year  # '개봉일'이 datetime 타입인지 확인

# 연도별 매출액 계산
yearly_revenue = df.groupby('년도')['매출액'].sum().reset_index()

# 그래프 생성
fig = go.Figure()

# 막대 그래프 추가
fig.add_trace(go.Bar(
    x=yearly_revenue['년도'],
    y=yearly_revenue['매출액'],
    marker_color='red'  # 막대 색상 변경
))

# 레이아웃 설정
fig.update_layout(
    title='연도별 매출액',
    xaxis=dict(title='년도'),
    yaxis=dict(title='매출액 (원)'),
    xaxis_tickangle=-45  # X축 레이블 각도 조정
)

# 그래프 표시
fig.show()


### 더 복합적 정보가 들어간 그래프

In [12]:
colors = plotly.colors.qualitative.Alphabet
colors

['#AA0DFE',
 '#3283FE',
 '#85660D',
 '#782AB6',
 '#565656',
 '#1C8356',
 '#16FF32',
 '#F7E1A0',
 '#E2E2E2',
 '#1CBE4F',
 '#C4451C',
 '#DEA0FD',
 '#FE00FA',
 '#325A9B',
 '#FEAF16',
 '#F8A19F',
 '#90AD1C',
 '#F6222E',
 '#1CFFCE',
 '#2ED9FF',
 '#B10DA1',
 '#C075A6',
 '#FC1CBF',
 '#B00068',
 '#FBE426',
 '#FA0087']

In [13]:
# 장르 목록
# df['genre_one'].unique()
genres = ['사극', '코미디', '판타지', '드라마', '액션', 'SF', '범죄', '전쟁', '미스터리', '어드벤처',
          '서부극(웨스턴)', '멜로/로맨스', '스릴러', '다큐멘터리', '공포(호러)', '애니메이션', '기타',
          '뮤지컬']

# Plotly의 Alphabet 컬러 팔레트
colors = plotly.colors.qualitative.Alphabet

# 각 장르에 색상 코드 할당
genre_colors = {genre: color for genre, color in zip(genres, colors)}

# '드라마' 장르 색상 변경 - Alphabet 팔레트의 다른 색상 사용
# 예를 들어 Alphabet 팔레트의 다음 색상을 사용
additional_color = '#FA0087'

# '드라마' 장르의 색상 변경
genre_colors['드라마'] = additional_color
genre_colors['미스터리'] = '#FBE426'
genre_colors


{'사극': '#AA0DFE',
 '코미디': '#3283FE',
 '판타지': '#85660D',
 '드라마': '#FA0087',
 '액션': '#565656',
 'SF': '#1C8356',
 '범죄': '#16FF32',
 '전쟁': '#F7E1A0',
 '미스터리': '#FBE426',
 '어드벤처': '#1CBE4F',
 '서부극(웨스턴)': '#C4451C',
 '멜로/로맨스': '#DEA0FD',
 '스릴러': '#FE00FA',
 '다큐멘터리': '#325A9B',
 '공포(호러)': '#FEAF16',
 '애니메이션': '#F8A19F',
 '기타': '#90AD1C',
 '뮤지컬': '#F6222E'}

- 아래 코드는 모든 연도를 통틀어 장르별 총 관객 수를 계산하여 장르를 정렬하고, 각 장르를 하나의 trace로 추가했다. 이렇게 하면 범례에 각 장르가 한 번씩만 나타나며, 모든 연도에 대해 같은 순서로 막대를 stack할 수 있다.
- 각 막대마다 장르순으로 stack하는 건 너무 많은 문제가 생겨서 포기

In [14]:
import plotly.graph_objects as go
import pandas as pd

# 데이터셋 로드 (다음 코드는 예제 데이터와 설정을 기반으로 함)
# df['개봉일'] = pd.to_datetime(df['개봉일'], format='%Y-%m-%d')
# df['년도'] = df['개봉일'].dt.year

# 연도별, 장르별 관객 수 및 영화 수 계산
grouped = df.groupby(['년도', 'genre_one'])
genre_yearly_data = grouped['관객수'].agg(['sum', 'count']).unstack(fill_value=0)

# 모든 연도를 통틀어 장르별 총 관객 수를 계산하여 장르를 정렬
overall_genre_order = genre_yearly_data['sum'].sum(axis=0).sort_values(ascending=False).index

# 그래프 객체 생성
fig = go.Figure()

# 정렬된 순서대로 막대 그래프 추가
for genre in overall_genre_order:
    fig.add_trace(go.Bar(
        x=genre_yearly_data['sum'].index,
        y=genre_yearly_data['sum'][genre],
        name=genre,
        marker_color=genre_colors.get(genre, '#000'),  # 색상 설정
        hoverinfo='text',  # 호버 정보 설정
        hovertext=[
            f"{year}년 <br>관객수 = {audience:,.0f}<br>장르 = {genre} ({genre_yearly_data['count'].loc[year, genre]}개)" 
            for year, audience in zip(genre_yearly_data['sum'].index, genre_yearly_data['sum'][genre])
        ]
    ))

# 레이아웃 업데이트
fig.update_layout(
    barmode='stack',
    title='연도별/장르별 총 관객 수',
    xaxis_title='연도',
    yaxis_title='관객 수',
    yaxis=dict(tickformat=','),
    legend_title='장르'
)

# 그래프 표시
fig.show()
fig.write_html('yj_year_genre.html')

## 2. 월별 장르별 총 관객수, 월별 각 장르의 영화 출품 수
어느 월에 어떤 장르의 영화가 잘 먹히는지(실제로 그 월에 잘 내고 있는지 확인)

In [15]:
# 'only_gener' 열에서 쉼표가 포함된 값들만 필터링하고 유니크한 값들을 가져오기
filtered_df = df[df['only_gener'].str.contains('기타')]

# 결과 출력
filtered_df

,순위,영화명,개봉일,매출액,관객수,스크린수,배급사,coach,actor_main_name,actor_sub_name,actor_cameo_name,maker,only_gener,genre_one,year,년도,월,일,년대
67,68,더 테러 라이브,2013-07-31,3.986671e+10,5583596,743,롯데쇼핑㈜롯데엔터테인먼트,김병우,"[하정우, 이경영, 전혜진, 이다윗]","[김홍파, 김대명, 최진호, 김소진, 강진아, 강신철, 한성천]",[],['씨네이천 2000'],"범죄, 기타",범죄,2013,2013,7,31,2010년대
116,117,콘크리트 유토피아,2023-08-09,3.741280e+10,3849242,1731,롯데컬처웍스롯데엔터테인먼트,엄태화,"[이병헌, 박서준, 박보영]","[김선영, 김도윤, 박지후]","[김학선, 공민정, 엄태구, 정영기, 오희준, 김준배]","['클라이맥스 스튜디오', '비에이치 엔터테인먼트']","드라마, 기타",드라마,2023,2023,8,9,2020년대
381,382,외계+인 2부,2024-01-10,1.379233e+10,1430121,1390,씨제이이엔엠,최동훈,"[류준열, 김태리, 김우빈, 이하늬, 염정아, 조우진, 김의성, 진선규]","[신정근, 윤경호, 이시훈]",[],['케이퍼필름'],기타,기타,2024,2024,1,10,2020년대


In [16]:
import plotly.graph_objects as go
import pandas as pd

# 데이터셋 로드 (예시 데이터셋 설정에 따라 로드)
# df['개봉일'] = pd.to_datetime(df['개봉일'], format='%Y-%m-%d')
# df['년도'] = df['개봉일'].dt.year

# 연대별로 데이터 필터링 및 그래프 생성
decades = {
    '1990년대': (1990, 1999),
    '2000년대': (2000, 2009),
    '2010년대': (2010, 2019),
    '2020년대': (2020, 2029)
}

for decade, (start_year, end_year) in decades.items():
    filtered_df = df[(df['년도'] >= start_year) & (df['년도'] <= end_year)]
    
    # 월별, 장르별 관객 수 계산
    grouped = filtered_df.groupby(['월', 'genre_one'])
    monthly_genre_data = grouped['관객수'].agg(['sum', 'count']).unstack(fill_value=0)

    # 모든 월을 통틀어 장르별 총 관객 수를 계산하여 장르를 정렬
    overall_genre_order = monthly_genre_data['sum'].sum(axis=0).sort_values(ascending=False).index

    # 그래프 객체 생성
    fig = go.Figure()

    # 정렬된 순서대로 막대 그래프 추가
    for genre in overall_genre_order:
        fig.add_trace(go.Bar(
            x=monthly_genre_data['sum'].index,
            y=monthly_genre_data['sum'][genre],
            name=genre,
            marker_color=genre_colors.get(genre, '#000'),  # 색상 설정
            hoverinfo='text',  # 호버 정보 설정
            hovertext=[
                f"{month}월 <br>관객수 = {audience:,.0f}<br>장르 = {genre} ({monthly_genre_data['count'][genre][month]}개)" 
                for month, audience in zip(monthly_genre_data['sum'].index, monthly_genre_data['sum'][genre])
            ]
        ))

    # 레이아웃 업데이트
    fig.update_layout(
        barmode='stack',
        title=f'월별/장르별 총 관객 수({decade})',
        xaxis_title='월',
        yaxis_title='관객 수',
        yaxis=dict(tickformat=','),
        legend_title='장르',
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(1, 13)),
            ticktext=[f"{m}월" for m in range(1, 13)]
        )
    )  

    # 그래프 표시
    fig.show()
    fig.write_html(f'yj_decade{decade}_genre.html')


# 다른 그래프 형태

## 트리맵 :
연도를 골랐을 때 그해에 영화별로 관객수가 얼마나 차지하는지 볼 수 있으면 좋겠습니다!

## 트리맵 URL 연결

In [17]:
# 년도 및 년대별로 가장 인기 있는 장르 계산
popular_genre_by_year = df.groupby(['년도', 'genre_one'])['관객수'].sum().unstack().idxmax(axis=1)
popular_genre_by_decade = df.groupby(['년대', 'genre_one'])['관객수'].sum().unstack().idxmax(axis=1)

# 년도별 색상을 가장 인기 있는 장르의 색상으로 지정
year_color = {year: genre_colors[genre] for year, genre in popular_genre_by_year.items()}
decade_color = {decade: genre_colors[genre] for decade, genre in popular_genre_by_decade.items()}

# URL 매핑 생성
urls = {}
for x in range(len(df)):
    name = df['영화명'].iloc[x]
    urls[name] = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={name}+영화'

# 트리맵 생성
fig = px.treemap(df, path=['년대', '년도', 'genre_one', '영화명'], values='관객수',
                 color='년도',  # '년도'로 색상 지정
                 color_discrete_map={**year_color, **decade_color, **genre_colors},  # 색상 매핑
                 hover_data=['영화명'])

# 커스텀 호버 템플릿 설정
fig.update_traces(hovertemplate="<b>%{label}</b><br>영화정보=%{id}<br>관객수=%{value}<extra></extra><br>영화명 %{customdata[0]}")

# 그래프 레이아웃 설정
fig.update_layout(
    autosize=False,
    width=1000,  # Adjust width as needed
    height=600   # Adjust height as needed
)

# fig에서 데이터 확인하여 영화명 순서 가져오기
temp = []

for label in fig.data[0].labels:
    temp.append(label)


# 영화명 순서에 맞게 URL 매핑하기
text_list = []
for movie_name in temp[:499]:
    if movie_name in urls:
        text_list.append(f'<a href="{urls[movie_name]}" target="_blank"><span style="color:white; font-size:10px;">Movie Link</span></a>')
    else:
        text_list.append(movie_name)
fig.update_traces(text=text_list)


app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Graph(id='treemap', figure=fig)
])

if __name__ == '__main__':
    app.run_server(debug=True)

fig.write_html('yj_treemap.html')
